<a href="https://colab.research.google.com/github/michalis0/Cloud-and-Advanced-Analytics/blob/main/2024/Week4-IoT/WalkThrough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a simple indoor/outdoor weather monitoring system

### Learning goals
By the end of this lab you will learn:
- How to setup an M5stack core2 device and connect it to Internet.
- How to program an M5stack device using the uiFlow platform (https://flow.m5stack.com/).
- How to insert new data into a bigQuery database.
- How to get current weather data for a specific location using the openweather api (https://home.openweathermap.org/).
- Creating a webapp with different endpoints using Flask and deploying it using GCloud cloud run.

-------------

# 1. **Introduction and IOT devices:**
Welcome to the Cloud and Advanced Analytics lab 4. In this lab, you will get know about M5stck IoT devices and sensors. You will program an M5stack device with an ENV-III sensor to build a simple weather station. Then you will push the weather data from the M5stack device to a BigQuery database in order to keep track of the historical weather data.


### IoT devices
In this lab you will work with the [M5stack Core2](https://docs.m5stack.com/en/core/core2) device and the [ENV-III unit](https://docs.m5stack.com/en/unit/envIII) which measures the temperature, humidity, etc.


<!-- <img src="./imgs/core2_01.webp" width="200"> <img src="./imgs/envIII_01.webp" width="200">  -->

<div class="row">
            <img src="https://drive.usercontent.google.com/download?id=110inQJbybfC9nBbJQj65fV0rPjKZ9tIk" width="300">
            <img src="https://drive.usercontent.google.com/download?id=19a_ROkvIBvTpgARpRr7b901_dKWLW7kH" width="300">
</div>

-------------

# **2. Setting up the weather sensor for measuring the indoor temperature and humidity**

In this part, following the instructions mentioned in the lecture, we will follow the following steps:
1. Connect the Core2 device to the power and connect it to the "iot-unil" WiFi network.
2. Connect the ENV-III sensor to the Core2 device.
3. Go to https://flow.m5stack.com/ and enter your device __API key__ in order to connect it to the device.
4. Write a micropython code script that gets the temperature and humidity from the ENV-III sensor and displays it on the device's screen. Use labels with proper names to indicate which value on the screen belongs to temperature and which one belongs to humidity. The temperature and humidity values should be updated every 1 minute.
  - Try to round the temperature and humidity values. You can do that by modifying the micropythin code in UIFlow.
5. Program the Core2 device with your code and check the result.

If everything goes in order, you should be able to see a result similar to the image below:

<img src="https://drive.usercontent.google.com/download?id=13YAw1-6T7GeZvnXLXCLpmTNUfhiiU5GK" width="300">

-------------

# 3. Creating a BigQuery database with the proper schema
In this part we will create a BigQuery database. First we have to define the schema of this database, i.e., what will be the columns of this database and what will be the data type of each column. Then we will see how we can add a new row to a database using a SQL query.

## 3.1 Defining a new BigQuery database
First, we need to define a new BigQuery database. This database will be later used to insert the historical weather data in it. Follow the steps below to create this database.

1. In your Google Cloud console, go to the BigQuery interface.
2. From the menu on the left select BigQuery Studio.

<img src="https://drive.usercontent.google.com/download?id=1EU4SxFTcuZRtevbF7N7VX9619EwrsiNi" width="300">

3. From the pannel on the left, click on the three dots next to your project name and select create dataset. This will create a new dataset in BigQuery.

<img src="https://drive.usercontent.google.com/download?id=1OGCVf1pbpFYEkN76XEC9Kj7_RRKgW3y2" width="300">

4. Assign a proper name to the dataset and select the region for the dataset.

<img src="https://drive.usercontent.google.com/download?id=1dYmV2NfikHufCPOUaZEYmKPVcPzXBH_8" width="300">

5. Create a new table in the new dataset. You can call it: **weather-records**

<img src="https://drive.usercontent.google.com/download?id=19kaG5orQd_JDjg_gPe4VW3LtPrBNKQUE" width="300">

6. We will create this table from an "Empty Table". Also we need to define the schema of this table. Define the schema according to the following image.

<img src="https://drive.usercontent.google.com/download?id=1GcuRVOHXjWMMbKFCjwZcfrOWs4eSIeJo" width="600">

## 3.2 Test the BigQuery database we just created (authentification Google Colab)
Now let's query the database we just created. The result of the query is expected to be an empty table

In [ ]:
# import the required packages in this walkthrough
import numpy as np
import pandas as pd
import requests
import time
from datetime import datetime
import os
import pandas as pd
import hashlib
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()
print("Authenticated")
# If you want to use your local jupyter notebook:
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "path-to-service-account-key-json"

Authenticated


In [ ]:
# Define the BigQuery client
client = bigquery.Client(project="YOUR-PROJECT-ID") # PROJECT_ID

In [ ]:

q = """
SELECT * FROM `PROJECT_ID.DATABAE_ID.weather-records`
LIMIT 10
"""

query_job = client.query(q)
df = query_job.to_dataframe()
df

## 3.3 Insert new rows in the BigQuery table
For this application, we need to insert new rows in the BigQuery table in order to be able to keep track of the historical data. To do this we can use the SQL `INSERT` statement. See the documentation to know more about the syntax of this statement: https://cloud.google.com/bigquery/docs/reference/standard-sql/dml-syntax#insert_statement

Note that if you do not specify any values for some of the columns, those columns will get a NULL value. Follow the following examples to better understand how does the `INSERT` statement works.

In [ ]:
# Insert some random values!
q = f"""
INSERT INTO `PROJECT_ID.DATABAE_ID.weather-records`
(date, time, indoor_temp, indoor_humidity)
VALUES('2024-03-15', '16:02:23', 23, 65)
"""

query_job = client.query(q)


In [ ]:
# Now we can query the database again and see the new row added to it!
# PROJECT_ID.DATABAE_ID.weather-records
q = """
SELECT * FROM `PROJECT_ID.DATABAE_ID.weather-records`
LIMIT 10
"""

query_job = client.query(q)
df = query_job.to_dataframe()
df

-------------

# 4. Creating a web service that receives data from the IoT device
Because of the limitations of the micropython, we are not able to install Google Cloud packages on the device and therefore we are not able to directly query the BigQuery table from the device. Therefore, we need to create a web service accesible via the Internet, so that we can send the data from the device to that web service. The job of the web service is to receive the data from the M5stack device and insert it in the BigQuery table.

The script `main.py` in the folder `web_app` will create a simple Flask application that can recieve data from the IoT device and insert it into your BigQuery database. pay attention to the following remarks about this flask app:

- A flask app in general can have multiple endpoint. In this part, we want to use the `/send-to-bigquery` endpoint. This endpoint will accept a post request from the IoT device that contains the indoor temperature values inside a json file. Then it builds a SQL query based on these values and sends this query to the BigQuery database. Please take a moment and look carefully how the SQL query is being made.
- It is important to make sure that only you, as the owner of the database, can call this endpoint and insert new rows inside the database! To do so, we need to make the calls to this endpoint __authenticated__. One way to do that is to set up a password which will be checked on the flask server. To avoid sending the password from the IoT device in plain text, we need to hash this password first and send it to the flask server. In the flask server, the correct hash string is already stored and each time a post request is made the received hash string (from the IoT device) will be checked against the stored hash string. In the `m5stack.py` script, see how we have set-up a password and converted it to a hash string. In the `main.py` script (flask app), see how the correctness of the hash string is checked for each post request. To know more about hash functions, check the [wikipedia page](https://en.wikipedia.org/wiki/Hash_function).
- In the `m5stack.py` script select a password for yourself (the `passwd` variable).
- In the `m5stack.py` script, observe that we send data to the flask app every 5 minutes.
- You need to store the hash of your password in the flask server. You can use the code cell below to hash your password. Copy the result to the `main.py` file (`YOUR_HASH_PASSWD` variable).

In [ ]:
h = hashlib.sha256(b"<YOUR_PASSWORD>")
h.hexdigest()

To test the endpoint locally on your computer, you can dowonload the `main.py` file and run it on your computer.

In your terminal run `python main.py`.

You can use the following cell to test your endpoint locally. __Note that the following cell will not work in colab since the service is now running only locally on your computer.__

In [ ]:
data = {
    "passwd": "YOUR-PASSWORD-HASH",
    "values": {
        "date": "<some_random_date>", # example of the correct format: '2024-02-22'
        "time": "<some_random_time>", # example of the correct format: '16:30:00'
        "indoor_temp": 23, # random values for the indoor temperature and humidity
        "indoor_humidity": 67
    }
}

requests.post("http://0.0.0.0:8080/send-to-bigquery", json=data)

If you have replaced the place holders with the correct string/values, then after running the above cell you should see the new row being added to your db.

In [ ]:

q = """
SELECT * FROM `PROJECT_ID.DATABAE_ID.weather-records`
LIMIT 10
"""

query_job = client.query(q)
df = query_job.to_dataframe()
df

-------------

# 5. Deploying the Flask app in Google Cloud

To deploy the Flask app in the Gcloud Cloud Run, you have to follow the following steps:
1. Download the following files from the course repository and push them to your private repository.
    - `main.py`
    - `requirements.txt`
    - `Dockerfile`
2. Navigate to Cloud Run in your Google Cloud console and open a cloud shell terminal.
3. In the terminal navigate to the directory corresponding to your git repository and do `git pull`
4. Follow the instructions in lab 1 to build the docker image and push it to the container registry.
5. Follow the instrucitons in lab 1 to deploy the created image in the container registry to Cloud Run.

After completing the above steps successfully, you would get a URL for Flask web service. Try to insert some artificial data to the BigQuery db using the URL. You can use the code below.

In [ ]:
data = {
    "passwd": "YOUR-PASSWORD-HASH",
    "values": {
        "date": "<some_random_date>", # example of the correct format: '2024-02-22'
        "time": "<some_random_time>", # example of the correct format: '16:30:00'
        "indoor_temp": 23, # random values for the indoor temperature and humidity
        "indoor_humidity": 67
    }
}

requests.post("<URL_FROM_CLOUDRUN>/send-to-bigquery", json=data)

Check if the data has been inserted correctly to the db.

In [ ]:
# PROJECT_ID.DATABAE_ID.weather-records
q = """
SELECT * FROM `PROJECT_ID.DATABAE_ID.weather-records`
LIMIT 10
"""

query_job = client.query(q)
df = query_job.to_dataframe()
df

-------------

# 6. Updating the `m5stack.py` with the URL of the deployed web service

Finally, you have to update the `m5stack.py` file with the URL of the web service you just deployed. After doing so, program the Core2 device with the new script. In the first iteration of the loop, the values of the ENV-III measurements will be sent to the server. So you sould be able to see the values added to the db almost immediately (You can check it by running a query!). After that, you should be able to see new values added to the server every 5 minutes!

-------------

# 7. Your Turn

1. Right now we are only inserting the measurements of the ENV-III sensor, temperature and humidity, to the db every 5 minutes. However, it is impossible to keep track of the historical data without keeping track of the actual time they were recorded. Therefore, we need to also send the time of the measurements from the Core2 device to the server. Try to do that! (Hint: check the `npttime` package: https://mpython.readthedocs.io/en/v2.2.1/library/micropython/ntptime.html)

2. In the Flask server, augment the `/send-to-bigquery` endpoint such that for each post request it gets the weather information from the openweather api and insert the corresponding values in the `outdoor_weather`, `outdoor_humidity`, and `outdoor_weather` columns of the db. For the `outdoor_weather` column, use the value corresponding to the key `description` from the json file you get from the openweather api.

3. Create a new endpoint `/get_outdoor_weather` in the Flask app that accepts post requests and sends back the outdoor humidity and temperature values it gets from the lastest entry in the db. For security reasons, to make sure that only you can see the records of your db, check the password in this method as well. Try to use this endpoint in the Core2 device so that you can show the outdoor weather info on the device.
    3.1. __bonus__: Can you also show the current date and time on the device?
    

After doing the 2nd and 3rd exercises, you can expect ot have this information on your Core2 device:

<img src="https://drive.usercontent.google.com/download?id=1fFtimdTCoP0GsL561hISafFbFe9fFeaP" width="500">

In [ ]:
# Help to call the openweather api

API_key = "YOUR_API_KEY"

city = "Lausanne"

url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_key}&units=metric'

response = requests.get(url)

In [ ]:
temp = round(response.json()['main']['temp'])
humidity = round(response.json()['main']['humidity'])
weather = response.json()['weather'][0]['description']

print(f"temp: {temp}, humidity: {humidity}, weather: {weather}")

temp: 9, humidity: 77, weather: overcast clouds
